In [176]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os, re
from numpy import genfromtxt
from PIL import Image, ImageFilter

In [3]:
data_dir = './data/'

In [312]:
sub_re = re.compile(r'^(.+)_Face-Threshold.*threshold.csv')
data_files = os.listdir(data_dir)
thresholds = []
ideal_threshold = {}
conditions = []
subjects =[]
for fname in data_files :
    subject = sub_re.findall(fname)
    if len(subject) > 0 :
        subject = subject[0]
        dat = pd.read_csv(data_dir + fname)
        condition = dat['Face'][0]
        thresholds.append(dat['Threshold'][0])
        subjects.append(subject)
        conditions.append(condition)
        if subject == 'ideal':
            ideal_threshold[condition] = dat['Threshold'][0]
efficiencies = np.zeros(len(thresholds))
for i in range(len(thresholds)):
    efficiencies[i] = thresholds[i] / ideal_threshold[conditions[i]]
results = pd.DataFrame({'Subject':subjects, 'Condition':conditions, 'Threshold':thresholds,
                        'Efficiency' : efficiencies})    
#human = results[results['Subject'] != 'ideal']
#sns.violinplot( y=human["Efficiency"], x=human["Condition"])
print(results)
#results.to_csv('Face-Threshold_results.csv')

  Subject  Condition  Threshold  Efficiency
0   ideal  Schematic   0.000685         1.0
1   ideal       Real   0.012242         1.0


In [302]:
subci_re = re.compile(r'^(.+)_classificationImage.*csv')
data_files = os.listdir(data_dir)
subjects =[]
for fname in data_files :
    subject = subci_re.findall(fname)
    if len(subject) > 0 :
        ci = genfromtxt(data_dir + fname, delimiter=',')
        ci = ci + np.abs(np.min(ci))
        ci = ci * (255 / np.max(ci))
        ci = np.floor(ci)
        ci_image = Image.fromarray(ci).convert('L')
        ci_image.show()
        ci_smooth = ci_image.filter(ImageFilter.GaussianBlur(4))
        ci_smooth.show()
        ci_threshold = np.array(ci_smooth)
        lb = np.quantile(ci_threshold, .025)
        ub = np.quantile(ci_threshold, .97)
        ci_threshold[ci_threshold < lb] = 0
        ci_threshold[ci_threshold > ub] = 255
        ci_threshold[np.logical_and(ci_threshold < ub,  ci_threshold > lb)] = 128
        ci_threshold = Image.fromarray(ci_threshold)
        ci_threshold.show()